![NumPy logo](img/numpylogo.svg)

# Miscellaneous and Advanced Topics

This module is a little bit of a grab bag of things that do not fit in easlier moduels, and most likely only somewhat advanced users need to worry about.

In [ ]:
import numpy as np
import numexpr as ne

## Record Arrays

We mention the use of records in NumPy for completeness.  In the last number of years, most of the purpose that would be served by records is better addressed with Pandas DataFrames.  However, Pandas is a very large and complex library, and it may be better to stay with raw NumPy for many purposes

An `np.record` is simply a way of collecting together several kinds of information within the same fixed-size data segment.  Record arrays are simply arrays, so all the same reshaping, slicing, filtering, broadcasting, etc. that works with other arrays works with record arrays.

Basic NumPy arrays always have elements of a single type.  However, that type can be a compound type.  The types of components of a record (or struct) can mostly be specified in two ways.  Using very short letter/number combinations, or using the NumPy type objects.

For example:

* b1, i1, i2, i4, i8, u1, u2, u4, u8, f2, f4, f8, c8, c16, a&lt;n&gt; (bytes, ints, unsigned ints, floats, complex and fixed length strings of a given *byte* lengths)
* `np.int8`, ..., `np.uint8`, ..., `np.float16`, `np.float32`, `np.float64`, `np.complex64`, etc. (similar but with *bit* sizes)

In [ ]:
# a record with a 4 byte int, a 4 byte float, 
# and 10 bytes of characters (ascii values)
arr = np.zeros((2,), dtype=('i4,f4,a10'))
arr

In [ ]:
arr[:] = [(1, 5., 'Hello'), (2, 6., 'World')]
arr

In [ ]:
# Access one field
arr['f1']

In [ ]:
# Perform a reduction on one field of the record array
arr['f1'].sum()

In [ ]:
# Update values within a field of a record array
arr['f1'][0] = 999
arr

## Saving and Loading Arrays

There are actually several different techniques for serializing and deserializing NumPy arrays.  At a basic level, Python `pickle` standard library module will work, but not as efficiently as some more specialized tools.

NumPy includes several mechanisms for saving internally.  We present the choice that is usually best, `np.savez_compressed()`, but be aware of `np.save()`, `np.savetxt()`, `np.savez()`.  These can be loaded with `np.load()` or `np.loadtxt()`.

However, for more high-performance needs—especially dealing with very large data—investigate [HDF5 for Python](https://www.h5py.org/) and [`joblib.dump()`](https://joblib.readthedocs.io/en/latest/generated/joblib.dump.html).

In [ ]:
%%file tmp/patient-records.csv
name,date,weight(kg),height(cm)
Alice,2011-01-01,85.1,170
Barb,2012-02-02,66.7,160
Carla,2013-03-03,29.5,120
Dagmar,2014-04-04,64.2,180

In [ ]:
patient_dtype = [("name", "a10"),
                 ("visit_date", 'datetime64[D]'),
                 ("weight", np.float32),
                 ("height", np.int16)]

data = np.loadtxt("tmp/patient-records.csv", 
                  skiprows=1, 
                  delimiter=",", 
                  dtype=patient_dtype,
                  converters = {1: np.datetime64})

In [ ]:
print("first row: ", data[0])
print("all weights: ", data['weight'])

In [ ]:
# BMI = kg / m**2
print("BMIs:", data['weight'] / (data['height']/100.0)**2)

In [ ]:
# Those patients with high BMI
print(data[(data['weight'] / (data['height']/100.0)**2) > 25])

While we illustrated I/O above using record arrays, there is nothing specific to those in the loading and saving mechanisms.  "Vanilla" NumPy arrays can equally be saved and loaded.

In [ ]:
arr1 = np.random.random(1000).reshape(10, 10, 10)
arr2 = np.random.randint(0, 100, 100).reshape(2, 5, 5, 2)
np.savez_compressed('tmp/random.npz', normal=arr1, int4d=arr2)

In [ ]:
data = np.load('tmp/random.npz')
print(data.files)
print(data['int4d'].shape)
print(data['normal'][7, 5, :])

## Compiled expressions

Operations on arrays peformed by NumPy are extremely fast, since they are written in optimized C or Fortran code (or sometimes use Numba to do just-in-time machine code generation with LLVM).

However, when you do a series of fast operations, you generally wind up round-tripping back to the much slower Python evaluation loop to dispatch the next operation.  The extra library `numexpr` can help reduce temporary arrays by compiling a string expression and returning a new array. It will also execute in parallel across many cores, where possible.

In [ ]:
import numexpr as ne

arr = np.random.random(100_000_000)
arr

In [ ]:
# Not necessarily a natural combination of operations, 
# but several manipulations to illustrate the point
%timeit np.sqrt(np.log((np.sin(arr**2) + np.cos(arr**3))))

In [ ]:
# Avoid 6 trips back-and-forth between array code and Python interpreter
%timeit ne.evaluate('sqrt(log((sin(arr**2) + cos(arr**3))))')

`numexpr` supports in-place. Be careful not to run this twice!

In [ ]:
out_arr = ne.evaluate('sin(arr**2) + cos(arr**3)', out=arr)
out_arr

In [ ]:
out_arr is arr

# Exercises

This module will ask you to perform several actions with a widely used dataset often provided as a machine learning example.  The UCI ML Breast Cancer Wisconsin (Diagnostic) dataset contains measures of tumors examined for malignancy.

We are not going to do machine learning per se in these exercises, but we would like to look at some summaries of the data.  The next cell will load the description of the data.  The actual dataset is contained in `data/wisconsin.csv`.

In [ ]:
# %load data/wisconsin.descr

Figure out how to load the data in a useful way using NumPy.  Then answer the following questions.  The answers are provided with question for you to check, but you need to arrive at them with your own code to verify.

In [ ]:
# Load the data
cancer = ...

Of those observations that have a larger than median value of "mean radius", what is the mean and standard deviation of their "concavity error"? 

Answers: 
* mean=0.036131897
* stddev=0.02302538

In [ ]:
# Mean/standard deviation of high median mean radius
...

Among the benign tumors, what is the correlation coefficient between "mean symmetry" and "mean fractal dimension" (i.e. Pearson product-moment correlation coefficient). 

Answer: 
* coefficient=0.41905971

In [ ]:
# Correlation coefficient between mean symmetry and mean fractal dimension
...

Which feature in the data shows the highest magnitude variance? 

Answer: worst area

In [ ]:
# Feature with highest variance
...

Which feature shows the highest normalized standard deviation?  I.e. standard deviation as a percentage of the entire value range of that feature.

Answer: benign?

In [ ]:
# Highest normalized standard deviation
...

You should verify the last answer; however, it is an artifact of the target field arbitrarily being encoded as 0/1 for False/True.  Disregarding the target, what field answers this question?

Answer: worst concave points

In [ ]:
# Highest normalized standard deviation
...